In [ ]:
!nvidia-smi

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as tt
from torchvision.transforms import Compose
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import os
import PIL
from PIL import Image

import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
# !pip install -U pandas-profiling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install -U featuretools

In [ ]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

# Exploring Dataset

In [ ]:
# Make copy of dataset to colab storage
!cp /content/drive/MyDrive/Category\ and\ Attribute\ Prediction\ Benchmark/Img/img.zip ./

In [ ]:
!unzip ./img.zip

In [ ]:
"""
- Fashion Landmark Annotations (Anno/list_landmarks.txt)
	fashion landmark labels. See LANDMARK LABELS section below for more info.

- Category Annotations (Anno/list_category_cloth.txt & Anno/list_category_img.txt)
	clothing category labels. See CATEGORY LABELS section below for more info.

- Attribute Annotations (Anno/list_attr_cloth.txt & Anno/list_attr_img.txt)
	clothing attribute labels. See ATTRIBUTE LABELS section below for more info.
"""

In [ ]:
# f = open("./list_category_img.txt", "r")
# print(f.readlines())

In [ ]:
!cp /content/drive/MyDrive/Category\ and\ Attribute\ Prediction\ Benchmark/Anno_coarse/list_category_img.txt ./
!cp /content/drive/MyDrive/Category\ and\ Attribute\ Prediction\ Benchmark/Anno_coarse/list_attr_img.txt ./

In [ ]:
import pandas as pd
df_attr = pd.read_csv("./list_category_img.txt", 
                       chunksize=10, 
                       sep = '\s+',
                       skiprows=[0,1],
                       names = ['Image','Category'])

result = df_attr.get_chunk(1000000)
result = result[result.Image != "img/Sheer_Pleated-Front_Blouse/img_00000001.jpg"]
print(result)

np_att = np.array(result)

In [ ]:
np_att[0][0]

In [ ]:
import PIL
from PIL import Image

for i in range(np_att.shape[0]):
    img_path = os.path.join('./', np_att[i+2][0])
    png = Image.open(img_path)
    category = np_att[i+2][1]
    # print(png.load()) 
    break

In [ ]:
png

In [ ]:
category

# Custom Dataset Loader

## Exploring attributes

In [ ]:
import pandas as pd
df_attr = pd.read_csv("./list_attr_img.txt", 
                       chunksize=10, 
                       sep = '\s+',
                       skiprows=[0,1],
                    #    names = ['Image','at']
                      )

result = df_attr.get_chunk(1000000)
print(result)

np_att = np.array(result)

In [ ]:
np_att[0]

In [ ]:
len(np_att[0][1:])
def one_hot_to_number(np_array):
    # out = np.where(np_array == 1)
    np_array[ np_array == -1] = 0
    return list(np_array)

len(one_hot_to_number(np_att[0][1:]))

In [ ]:
dataset_pairs = {}
for idx in range(np_att.shape[0]):
    dataset_pairs[np_att[idx][0]] = one_hot_to_number(np_att[idx][1:])

In [ ]:
# Custom dataset for zipping input and output images
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, combine_dir, transform, transform_size=32, data_len=1000000):
        # One Folder with training inputs and outputs 
        self.combine_dir = combine_dir
        self.both_dir = os.listdir(combine_dir)
        self.transform = transform
        self.input_dir = './'
        self.dataset_pairs = dataset_pairs
        all_imgs = []
        category = []

        # Category 
        df_category = pd.read_csv("./list_category_img.txt", 
                       chunksize=10, 
                       sep = '\s+',
                       skiprows=[0,1],
                       names = ['Image','Category'])
        result = df_category.get_chunk(data_len)
        result = result[result.Image != "img/Sheer_Pleated-Front_Blouse/img_00000001.jpg"]
        self.np_cat = np.array(result)
        # Deleting one extra element
        # self.np_cat = np.delete(self.np_cat, 'img/Sheer_Pleated-Front_Blouse/img_00000001.jpg')

        for i in range(self.np_cat.shape[0]):
            all_imgs.append(os.path.join('./', self.np_cat[i][0]))
            category.append(self.np_cat[i][1])
        

        # Attribute
        # df_attr = pd.read_csv("./list_attr_img.txt", 
        #                chunksize=10, 
        #                sep = '\s+',
        #                skiprows=[0,1],
        #             #    names = ['Image','Category']
        #               )

        # result = df_attr.get_chunk(1000000)
        # np_att = np.array(result)

        
        # for idx in range(np_att.shape[0]):
        #     self.dataset_pairs[np_att[idx][0]] = one_hot_to_number(np_att[idx][1:])


        self.total_imgs  = all_imgs #natsort.natsorted(all_imgs)
        self.totla_cate  = category

    def __len__(self):
        return len(self.total_imgs)

    def one_hot_to_number(np_array):
        # out = np.where(np_array == 1)
        np_array[ np_array == -1] = 0
        return list(np_array)

    def __getitem__(self, idx):
        # Load input images
        # if self.np_cat[idx][0] in self.dataset_pairs.keys():
        # 'img/Sheer_Pleated-Front_Blouse/img_00000001.jpg'
        in_img_loc = os.path.join(self.input_dir, self.total_imgs[idx])
        input_image = Image.open(in_img_loc).convert("RGB")
        input_tensor = self.transform(input_image)

        # Load output catogery
        catogery_label = torch.tensor(self.totla_cate[idx])

        # Load output Attribute of same image
        att_label = torch.tensor(self.dataset_pairs[self.np_cat[idx][0]])

        return input_tensor, catogery_label, att_label

In [ ]:
size = 256
batch_size = 64
dataset = CustomDataSet("./", 
                        transform=Compose([ 
                        tt.Resize((size,size),interpolation=Image.ANTIALIAS),
                        tt.ToTensor()]),
                        transform_size=size,
                        data_len=1000000)

In [ ]:
len(dataset.np_cat[:,0]), len(dataset.dataset_pairs.keys())

In [ ]:
# Dowloading dataset and defining dataloader
train_loader = torch.utils.data.DataLoader(dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2,
                                           pin_memory=True)

In [ ]:
# Validation DataLoader
val_dataset = CustomDataSet("./", 
                        transform=Compose([ 
                        tt.Resize((size,size),interpolation=Image.ANTIALIAS),
                        tt.ToTensor()]),
                        transform_size=size,
                        data_len=10000)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                           batch_size=batch_size*4,
                                           shuffle=True,
                                           num_workers=2,
                                           pin_memory=True)

In [ ]:
# Define function to display dataset
def denorm(img_tensor):
    return img_tensor

def show_images(images):
    fig, ax = plt.subplots(figsize=(12,12))
    ax.set_xticks([]); ax.set_yticks([]);
    print(images.shape)
    ax.imshow( make_grid( denorm(images.detach()[:64]), nrow=8).permute((1,2,0)))
    

def show_batch(data_loader):
    for images, category, attribute in data_loader:
        show_images(images)
        return category, attribute

def show_output(data_loader):
    for _, images in data_loader:
        show_images(images)
        break;

In [ ]:
# Display training samples
category, attribute = show_batch(train_loader)

In [ ]:
category.shape, attribute.shape

In [ ]:
# Setup GPU configuration and DeviceDataLoader for efficient GPU memory usage
def get_default_device():
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def to_device(data, device):
    ''' Loading data to device '''
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in  data]
    else:
        return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.device)

    def __len__(self):
        return len(self.dl)

# Model Implementation

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# from vit_pytorch import ViT

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def accuracy2(outputs, labels):
    # gt_S  =np.asarray(gt_S)
    outputs = F.sigmoid(outputs) 
    # print(outputs.shape)
    outputs = torch.round(outputs)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))
    # return torch.tensor(torch.sum(outputs*labels).item() / int(torch.sum(labels*labels).item()))


class Base(nn.Module):
    def training_step(self, batch):
        input, category, attribute = batch
        out_category, out_attribute = self(input)                  # Generate predictions
        attribute = attribute.type_as(out_attribute)
        loss1 = 100*F.cross_entropy(out_category, category)  # Calculate loss
        loss2 = F.binary_cross_entropy_with_logits(out_attribute, attribute)
        return loss1 + loss2

    def validation_step(self, batch):
        input, category, attribute = batch
        out_category, out_attribute = self(input)                    # Generate predictions
        attribute = attribute.type_as(out_attribute)
        loss1 = F.cross_entropy(out_category, category)  # Calculate loss
        loss2 = 100*F.binary_cross_entropy_with_logits(out_attribute, attribute)
        acc = accuracy(out_category, category)           # Calculate accuracy
        acc2 = accuracy2(out_attribute, attribute)           # Calculate accuracy
        return {'val_loss': loss1.detach() + loss2.detach(), 'val_acc': acc, 'val_acc2': acc2}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        # Attribute acc
        batch_accs2 = [x['val_acc2'] for x in outputs]
        epoch_acc2 = torch.stack(batch_accs2).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(), 'val_acc2': epoch_acc2.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}],{} train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, val_acc2: {:.4f}".format(
            epoch, "last_lr: {:.5f},".format(result['lrs'][-1]) if 'lrs' in result else '', 
            result['train_loss'], result['val_loss'], result['val_acc'], result['val_acc2']))

In [ ]:
class Model(Base):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.resnext50_32x4d(pretrained=True) #torchvision.models.wide_resnet101_2(pretrained=True)
        
        self.model.fc = nn.Sequential( nn.Linear(2048, 2048),
                                       nn.ReLU(inplace=True))

        self.category_head = nn.Sequential( nn.Linear(2048, 50))
        self.attribute_head = nn.Sequential( nn.Linear(2048, 1000))

    def forward(self, images):
        out = self.model(images)
        pred_category = self.category_head(out)
        pred_attribute = self.attribute_head(out)

        return pred_category, pred_attribute



In [ ]:
import torch
from tqdm.notebook import tqdm

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, optimizer,opt_func=torch.optim.SGD):
    history = []
    # optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_loader, device)
valid_dl = DeviceDataLoader(val_loader, device)


In [ ]:
model = Model()
to_device(model, device)

# Evaluate
# evaluate(model, valid_dl)

In [ ]:
epochs = 6
max_lr = 0.01
grad_clip = None#0.1
weight_decay = 0#1e-4
opt_func = torch.optim.Adam

In [ ]:
# %%time
history = fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

In [ ]:
lr = 0.0015
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
history += fit(epochs, lr, model, train_dl, valid_dl , optimizer)

In [ ]:
lr = 0.0002
history += fit(epochs, lr, model, train_dl, valid_dl , optimizer)

In [ ]:
states = {
            'epoch': epochs + 1,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
torch.save(states, "./resnext_6.h5")